In [3]:
from MQGO import data, main, ML_engine, optimizer, QC_engine 
from deepmd.calculator import DP
import subprocess
import numpy as np
import json

In [4]:
d_steps = [5000]
tot_steps = [20000]
subprocess.run('cp water1.xyz water.xyz', shell=True)

index = 0
for ii in range(0,len(d_steps)):
    for jj in range(0,len(tot_steps)):
        js_setting = {}
        with open('./input0.json', 'r') as file:
            json_setting = json.load(file)
            print('after remake json: ',json_setting['learning_rate']['decay_steps'])
            
        json_setting['learning_rate']['decay_steps'] = d_steps[ii]
        json_setting['learning_rate']['start_lr'] = 0.005
        json_setting['training']['numb_steps'] = tot_steps[jj]
        print('ii,jj:',ii,jj)
        print('before make json: ',json_setting['learning_rate']['decay_steps'])
        with open('./input.json', 'w') as js:
            json.dump(json_setting, js, indent=4)
        
        setting = {'basis':'6-31g', 'spin':0, 'xc':None, 'restricted':True, 'verbose':3, 'max_cycle':500}
        geo = main.MLgeomopt(qc_engine='pyscf', ml_engine='deepmd', work_path='./', xyz_path='./water.xyz',**setting)
        geo.opt_algorithm = 'lbfgs'
        geo.debug = True
        geo.have_init_qc_data = False
        geo.kernel()
        
        Opt = optimizer.Optimizer(xyz_path="water0.xyz",
									work_path="./",
									ml_engine='deepmd',
									outter_cycle=1,
									algorithm='lbfgs',
									max_opt_cycle=100)
        atom0 = Opt.ase_read_xyz()
        atom0.calc = DP(model='graph.pb')
        print(atom0.get_positions())
        force0_ml = atom0.get_forces()

        forces = np.loadtxt('raw/force.raw')
        force0_qc = forces[0].reshape((6,3))

        delta = force0_qc-force0_ml
        with open('debug_diff_force.txt', 'a') as file:
            file.write('%24.18f\n'%atom0.get_potential_energy())
            for i in range(0, len(force0_qc)):
                for j in range(0, 3):
                    file.write('%24.18f'%delta[i][j])
                file.write('\n')
            file.write('\n')

        subprocess.run('cp water1.xyz water.xyz', shell=True)
        with open('./debug.txt', 'a') as f:
            f.write('\n')

after remake json:  4000
ii,jj: 0 0
before make json:  5000
converged SCF energy = -151.962995631973
--------------- RHF gradients ---------------
         x                y                z
0 O    -0.0341709105     0.0193993478     0.0404963135
1 O     0.0341709105     0.0193993478    -0.0404963135
2 H     0.0260049036     0.0097506786    -0.0219203401
3 H     0.0079828195    -0.0291500263    -0.0185751130
4 H    -0.0079828195    -0.0291500263     0.0185751130
5 H    -0.0260049036     0.0097506786     0.0219203401
----------------------------------------------
--------------- RHF gradients ---------------
         x                y                z
0 O    -0.0341709105     0.0193993478     0.0404963135
1 O     0.0341709105     0.0193993478    -0.0404963135
2 H     0.0260049036     0.0097506786    -0.0219203401
3 H     0.0079828195    -0.0291500263    -0.0185751130
4 H    -0.0079828195    -0.0291500263     0.0185751130
5 H    -0.0260049036     0.0097506786     0.0219203401
----------

2022-05-18 11:08:56.647844: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-18 11:08:56.647981: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Traceback (most recent call last):
  File "/home/dgg95223/venv/bin/dp", line 5, in <module>
    from deepmd.entrypoints.main import main
  File "/home/dgg95223/venv/lib/python3.7/site-packages/deepmd/__init__.py", line 7, in <module>
    import deepmd.utils.network as network
  File "/home/dgg95223/venv/lib/python3.7/site-packages/deepmd/utils/__init__.py", line 2, in <module>
    from .data import DeepmdData
  File "/home/dgg95223/venv/lib/python3.7/site-packages/deepmd/utils/data.py", line 10, in <module>
    from deepmd.env import GLOBAL_NP_FLOAT_PRECISION
  File "/home/dgg95223/venv/lib/python3.7/site

KeyboardInterrupt: 

In [4]:
a,b,c = data.read_xyz('water.xyz',index=1,output='regular')

In [ ]:
mo = np.loadtxt('water.xyz',usecols=(1,2,3),skiprows=10)
# print(mo[1])
atoms =  ''
for i in range(0, len(b)):
    _mo = ''
    for j in range(0, 3):
        _mo += str(mo[i][j]) + '  '
    atoms += b[i] + '  ' + _mo + '\n'
print(atoms)

In [35]:
from pyscf import gto, scf, geomopt
from pyscf.data.nist import BOHR, HARTREE2EV
mol = gto.Mole(atom=atoms, basis='6-31g', spin=0).build()
mf = scf.RHF(mol)
mf.max_cycle = 500
mf.kernel()
print(mf.e_tot * HARTREE2EV)

converged SCF energy = -150.91286083677
-4106.548111611891


In [36]:
mol_eq = mf.Gradients().optimizer(solver='berny').kernel()

INFO:pyscf.geomopt.berny_solver.139766248749424:0 Internal coordinates:
INFO:pyscf.geomopt.berny_solver.139766248749424:0 * Number of fragments: 5
INFO:pyscf.geomopt.berny_solver.139766248749424:0 * Number of internal coordinates: 17
INFO:pyscf.geomopt.berny_solver.139766248749424:0 * Number of strong bonds: 1
INFO:pyscf.geomopt.berny_solver.139766248749424:0 * Number of weak bonds: 6
INFO:pyscf.geomopt.berny_solver.139766248749424:0 * Number of weak angles: 4
INFO:pyscf.geomopt.berny_solver.139766248749424:0 * Number of superweak angles: 6



Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O  -0.446142   1.122648   0.305449    0.000000  0.000000  0.000000
   O   0.446142   1.122648  -0.305449    0.000000  0.000000  0.000000
   H   1.120019   1.122057   1.634793    0.000000  0.000000  0.000000
   H   0.424495  -2.593506   6.209108    0.000000  0.000000  0.000000
   H  -0.424495  -2.593506  -6.209108    0.000000  0.000000  0.000000
   H  -1.120019   1.122057  -1.634793    0.000000  0.000000  0.000000
converged SCF energy = -150.912860849479
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 O     0.2725308556     0.0000039204    -0.1866346836
1 O    -0.2725327075     0.0000045406     0.1866358753
2 H     0.0013679178     0.0000026531     0.0019656291
3 H     0.0000603113    -0.0000074205     0.0009091055
4 H    -0.0000603094    -0.0000074272    -0.0009091041
5 H    -0.0013660678     0.0000037336

INFO:pyscf.geomopt.berny_solver.139766248749424:1 Energy: -150.912860849
INFO:pyscf.geomopt.berny_solver.139766248749424:1 Pseudoinverse gap of only: 1.8e+05
INFO:pyscf.geomopt.berny_solver.139766248749424:1 Minimization on sphere was performed:
INFO:pyscf.geomopt.berny_solver.139766248749424:1 * Trust radius: 0.3
INFO:pyscf.geomopt.berny_solver.139766248749424:1 * Number of negative eigenvalues: 0
INFO:pyscf.geomopt.berny_solver.139766248749424:1 * Lowest eigenvalue: 7.08e-05
INFO:pyscf.geomopt.berny_solver.139766248749424:1 * lambda: -0.513
INFO:pyscf.geomopt.berny_solver.139766248749424:1 Quadratic step: RMS: 0.0728, max: 0.273
INFO:pyscf.geomopt.berny_solver.139766248749424:1 * Predicted energy change: -0.0682
INFO:pyscf.geomopt.berny_solver.139766248749424:1 Total step: RMS: 0.0728, max: 0.273
INFO:pyscf.geomopt.berny_solver.139766248749424:1 Perfect transformation to cartesians in 9 iterations
INFO:pyscf.geomopt.berny_solver.139766248749424:1 * RMS(dcart): 2.53e-07, RMS(dq): 3.38


Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O  -0.524816   1.092441   0.314712   -0.078674 -0.030208  0.009262
   O   0.524818   1.092436  -0.314711    0.078677 -0.030212 -0.009262
   H   1.009088   1.184344   1.681556   -0.110931  0.062287  0.046763
   H   0.464547  -2.625579   6.197497    0.040052 -0.032073 -0.011610
   H  -0.464549  -2.625579  -6.197501   -0.040054 -0.032073  0.011607
   H  -1.009089   1.184337  -1.681553    0.110930  0.062280 -0.046760

WARN: Large deviations found between the input molecule and the molecule from chkfile
Initial guess density matrix may have large error.

converged SCF energy = -150.945371540259
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 O    -0.0271851711    -0.0009128467     0.0162616601
1 O     0.0271855915    -0.0009135077    -0.0162612143
2 H     0.0026043494     0.0009139346     0.0043144030
3 H     

INFO:pyscf.geomopt.berny_solver.139766248749424:2 Energy: -150.94537154
INFO:pyscf.geomopt.berny_solver.139766248749424:2 Hessian update information:
INFO:pyscf.geomopt.berny_solver.139766248749424:2 * Change: RMS: 0.0434, max: 0.282
INFO:pyscf.geomopt.berny_solver.139766248749424:2 Trust update: Fletcher's parameter: 0.477
INFO:pyscf.geomopt.berny_solver.139766248749424:2 Linear interpolation:
INFO:pyscf.geomopt.berny_solver.139766248749424:2 * Energies: -150.94537, -150.91286
INFO:pyscf.geomopt.berny_solver.139766248749424:2 * Derivatives: -0.00813, 0.0902
INFO:pyscf.geomopt.berny_solver.139766248749424:2 * Quartic interpolation was performed: t = 0.142
INFO:pyscf.geomopt.berny_solver.139766248749424:2 * Interpolated energy: -150.94597
INFO:pyscf.geomopt.berny_solver.139766248749424:2 Minimization on sphere was performed:
INFO:pyscf.geomopt.berny_solver.139766248749424:2 * Trust radius: 0.3
INFO:pyscf.geomopt.berny_solver.139766248749424:2 * Number of negative eigenvalues: 0
INFO:pys

...


Geometry optimization cycle 62
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.164232   4.457047   0.583190    0.027209 -0.027193 -0.065325
   O   0.151245   4.447623  -0.612653   -0.091506 -0.015470  0.027677
   H   0.140940   1.544413  -0.093449    0.031521 -0.015156  0.013092
   H   1.293018  -6.405298  -0.385759    0.257432  0.039908 -0.049580
   H  -1.201346  -6.374190   0.390625   -0.281753  0.045349  0.037051
   H  -0.548090   1.632803   0.118045    0.057098 -0.027437  0.037085

WARN: Large deviations found between the input molecule and the molecule from chkfile
Initial guess density matrix may have large error.

converged SCF energy = -151.43379615614
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 O    -0.0001055252    -0.0011163238     0.0048820027
1 O    -0.0001875197    -0.0012152624    -0.0048580714
2 H    -0.0019915948     0.0014696139     0.0006223835
3 H     

INFO:pyscf.geomopt.berny_solver.139766248749424:62 Energy: -151.433796156
INFO:pyscf.geomopt.berny_solver.139766248749424:62 Hessian update information:
INFO:pyscf.geomopt.berny_solver.139766248749424:62 * Change: RMS: 3.27e+04, max: 1.4e+05
INFO:pyscf.geomopt.berny_solver.139766248749424:62 Trust update: Fletcher's parameter: -5.56e+03
INFO:pyscf.geomopt.berny_solver.139766248749424:62 Linear interpolation:
INFO:pyscf.geomopt.berny_solver.139766248749424:62 * Energies: -151.4338, -151.71392
INFO:pyscf.geomopt.berny_solver.139766248749424:62 * Derivatives: 5e-06, 8.34e-06
INFO:pyscf.geomopt.berny_solver.139766248749424:62 * Cubic interpolation was performed: t = 1.0
INFO:pyscf.geomopt.berny_solver.139766248749424:62 * Interpolated energy: -151.71392
INFO:pyscf.geomopt.berny_solver.139766248749424:62 Minimization on sphere was performed:
INFO:pyscf.geomopt.berny_solver.139766248749424:62 * Trust radius: 4.7e-06
INFO:pyscf.geomopt.berny_solver.139766248749424:62 * Number of negative eige


Geometry optimization cycle 63
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.164202   4.457046   0.583199   -0.000031 -0.000000  0.000008
   O   0.151273   4.447623  -0.612645    0.000029 -0.000000  0.000007
   H   0.140946   1.544413  -0.093450    0.000006 -0.000000 -0.000001
   H   1.293028  -6.405297  -0.385780    0.000010  0.000001 -0.000020
   H  -1.201357  -6.374189   0.390651   -0.000011  0.000001  0.000026
   H  -0.548092   1.632801   0.118026   -0.000002 -0.000001 -0.000020
converged SCF energy = -151.433793116824
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 O    -0.0001079835    -0.0011162774     0.0048824408
1 O    -0.0001850590    -0.0012153107    -0.0048585231
2 H    -0.0019902156     0.0014694454     0.0006219012
3 H     0.0460884107    -0.0005746361    -0.0143460005
4 H    -0.0460885031     0.0005746666     0.0143460262
5 H     0.0022833504     0.000862112

INFO:pyscf.geomopt.berny_solver.139766248749424:63 Energy: -151.433793117
INFO:pyscf.geomopt.berny_solver.139766248749424:63 Hessian update information:
INFO:pyscf.geomopt.berny_solver.139766248749424:63 * Change: RMS: 8.49e+05, max: 3.63e+06
INFO:pyscf.geomopt.berny_solver.139766248749424:63 Trust update: Fletcher's parameter: -1.46
INFO:pyscf.geomopt.berny_solver.139766248749424:63 Linear interpolation:
INFO:pyscf.geomopt.berny_solver.139766248749424:63 * Energies: -151.43379, -151.71392
INFO:pyscf.geomopt.berny_solver.139766248749424:63 * Derivatives: 1.96e-06, 2.08e-06
INFO:pyscf.geomopt.berny_solver.139766248749424:63 * Cubic interpolation was performed: t = 1.0
INFO:pyscf.geomopt.berny_solver.139766248749424:63 * Interpolated energy: -151.71392
/home/jingheng/anaconda3/lib/python3.9/site-packages/berny/Math.py:135: RuntimeWarning: divide by zero encountered in double_scalars
  x = x - fx / dxf
/home/jingheng/anaconda3/lib/python3.9/site-packages/berny/berny.py:243: RuntimeWarning

FindrootException: 

In [37]:
print(-151.433793117*HARTREE2EV)

-4120.723400979506


In [38]:
from ase.io import read
a = read('./water.xyz', index=':')
print(a[0].get_positions())

[[ 3.63802  1.5711   1.76   ]
 [26.36198  1.5711   1.76   ]
 [ 3.95828  2.55412  1.74392]
 [ 2.63613  1.66728  1.76493]
 [26.09174  0.99418  2.56104]
 [26.0955   0.9908   0.96072]]


In [5]:
import numpy as np
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

with open('./debug_copy.txt', 'r') as file:
    results = file.readlines()

data = []
_data = []
for ii, i in enumerate(results):
    if i != '\n':
        _data_ = i.split()[0:2]
        _data.append(_data_)   
    elif i == '\n':
        data.append(np.array(_data, dtype=np.float64))
        _data = []

fig,axs = plt.subplots(figsize=(16,9))
for ii, i in enumerate(data):
    axs.plot(list(range(1, len(i)+1)),i[:,0], label='QC '+str(ii+1)) # QC
    axs.plot(list(range(1, len(i)+1)),i[:,1], label='ML '+str(ii+1)) # ML

# plt.rc('xtick', labelsize=18)
# plt.rc('ytick', labelsize=18)
axs.set_xlabel('Cycle', fontsize=20)
axs.set_ylabel('Total Energy (eV)', fontsize=20)
axs.set_xlim(0, 53)
axs.set_ylim(-4137, -4130)
axs.set_title('QC energy VS ML energy', y=1.05, fontsize=24)
axs.legend(fontsize=11)
    

plt.tight_layout()
plt.savefig('comparsion.pdf')